In [3]:
#import processing
#import ds
#import ml
#import model_saving
#import TS
#import TA
# Load the necessary packages and modules
#from pandas_datareader import data as pdr
#import matplotlib.pyplot as plt
import pandas as pd
import numpy
import numpy as np
import math as m
#import talib
import pickle
#from sklearn import svm

#from hmmlearn.hmm import GaussianHMM
#from sklearn.externals import joblib

In [4]:
filename='/home/octo/Dropbox'+ '/SPY24.csv'

In [10]:
# Import a Kalman filter and other useful libraries
from pykalman import KalmanFilter
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
from scipy import poly1d

def kalman_ma(data):
    x=data.Open
    # Construct a Kalman filter
    kf = KalmanFilter(transition_matrices = [1],
                  observation_matrices = [1],
                  initial_state_mean = x[1],
                  initial_state_covariance = 1,
                  observation_covariance=1,
                  transition_covariance=.01)

    # Use the observed values of the price to get a rolling mean
    state_means, _ = kf.filter(x.values)
    state_means = pd.Series(state_means.flatten(), index=x.index)
    #df_ml['km']=state_means
    return state_means

In [11]:
# loading csv file
def get_csv_pd(path):
    spy_pd=pd.read_csv(filename,sep=',',dtype={'Stock':np.str,'Close':np.float32,'High':np.float32,'Low':np.float32,'askSize':np.float32,'askPrice':np.float32,'Volume':np.float32,
                                           'bidPrice':np.float32,'bidSize':np.float32,'lastprice':np.float32},index_col=0,parse_dates=True)
    return spy_pd
def processing(df):
    #df=df.drop(['Stock'], axis=1)
    df['Open']=((df.askPrice+df.bidPrice)/2).shift(-1)
    df['vwap']=((df.loc[:,'bidPrice']*df.loc[:,'bidSize'])+(df.loc[:,'askPrice']*df.loc[:,'askSize']))/(df.loc[:,'bidSize']+df.loc[:,'askSize'])
    df['km']=kalman_ma(df)
    df['OC']=(df.Open-df.Close)
    df['HO']=(df.High-df.Open)
    df['OL']=(df.Open-df.Low)
    df['UL']=df.askPrice.rolling(20).max()
    df['LL']=df.bidPrice.rolling(20).min()
    #df['state']=np.where(np.logical_and(df.km>df.UL,df.bidSize>50),1,
    #                 np.where(np.logical_and(df.km<df.LL,df.askSize>50),-1,0))
    #df=df.drop(['Volume','High','Low','Close','askPrice','bidPrice','askSize','bidSize','lastprice'], axis=1)
    return df
def normalize(df):
    df.Open=df.Open/df.Open.rolling(20).mean()## target
    df.vwap=df.vwap/df.lastprice.rolling(20).mean()
    df.OC=df.OC/df.OC.rolling(20).mean()
    df.HO=df.HO/df.HO.rolling(20).mean()
    df.OL=df.OL/df.OL.rolling(20).mean()
    df.UL=df.UL/df.lastprice.rolling(20).mean()
    df.LL=df.LL/df.lastprice.rolling(20).mean()
    df.km=df.km/df.lastprice.rolling(20).mean() 
    df.Volume=df.Volume/df.Volume.rolling(20).mean() 
    return df

In [35]:
data=get_csv_pd(filename)
data=processing(data)
data=normalize(data)

In [87]:
dataset=data[['Open','vwap','OC','HO','OL','UL','LL','km','Volume']]

In [88]:
dataset.dtypes

Open      float64
vwap      float64
OC        float64
HO        float64
OL        float64
UL        float64
LL        float64
km        float64
Volume    float64
dtype: object

In [89]:
#dataset=dataset[500:200000]
dataset=dataset[2500:35000]

In [90]:
dataset.tail()

,Open,vwap,OC,HO,OL,UL,LL,km,Volume
2018-04-24 20:50:48.881208,0.999973,0.999970,1.026639,1.004318,0.988228,1.000053,0.999865,0.999972,1.000011
2018-04-24 20:50:48.885337,0.999977,0.999974,1.021951,1.003572,0.990235,1.000056,0.999869,0.999972,1.000009
2018-04-24 20:50:48.889516,0.999981,0.999977,1.018232,1.002976,0.991847,1.000060,0.999872,0.999973,1.000008
2018-04-24 20:50:48.893596,0.999985,0.999981,1.014539,1.002380,0.993464,1.000064,0.999876,0.999975,1.000019
2018-04-24 20:50:48.897602,0.999989,0.999956,1.010873,1.001785,0.995086,1.000068,0.999880,0.999976,1.000017


## Below 1 and <previous == sell along with classification signal+ mean of last two open price

In [91]:
dataset=dataset.dropna()
dataset=dataset.values
dataset = dataset.astype('float32')

In [92]:
# fix random seed for reproducibility
numpy.random.seed(107)

In [93]:
dataset[3]

array([0.9999812, 0.9999782, 1.0238303, 1.0030894, 0.9666328, 1.0000507,
       0.9999756, 0.9999975, 1.0000194], dtype=float32)

In [23]:
#from sklearn.preprocessing import MinMaxScaler
# normalize the dataset
#scaler = MinMaxScaler(feature_range=(0, 1))
#dataset = scaler.fit_transform(dataset)

In [94]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        b = dataset[i:(i+look_back), 1]
        c = dataset[i:(i+look_back), 2]
        d = dataset[i:(i+look_back), 3]
        e=  dataset[i:(i+look_back), 4]
        f = dataset[i:(i+look_back), 5]
        g=  dataset[i:(i+look_back), 6]
        h=  dataset[i:(i+look_back), 7]
        j=  dataset[i:(i+look_back), 8]
        dataX.append(np.c_[b,c,d,e,f,g,h,j])
        #dataX.append(b)
        #dataX.append(c)
        #dataX.append(d)
        #dataX.append(e)
        #dataX.concatenate((a,bT,cT,dT,eT),axis=1)
        dataY.append(dataset[i + look_back,0])
        #dataY.append(a)
    return np.array(dataX), np.array(dataY)

In [95]:
# reshape into X=t and Y=t+1
look_back =4
X, Y = create_dataset(dataset,look_back)

In [96]:
X.shape

(32494, 4, 8)

In [97]:
X.shape

(32494, 4, 8)

In [78]:
epochs=3
batch_size=25

In [31]:
import keras
keras.__version__

Using TensorFlow backend.


'2.1.6'

In [48]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

In [86]:
## First Model

In [79]:
# create and fit the LSTM network
model = Sequential()

In [99]:
#model.add(LSTM(6,input_shape=(X.shape[1],X.shape[2])))
#model.add(Dense(1))

In [100]:
#model.compile(loss='mean_squared_error', optimizer='adam')
#model.fit(X,Y, epochs, batch_size, verbose=2)

## model 2
https://faroit.github.io/keras-docs/1.0.1/getting-started/sequential-model-guide/
https://keras.io/getting-started/sequential-model-guide/

In [101]:
# create and fit the LSTM network
model = Sequential()

In [57]:
X.shape

(32489, 9, 8)

In [102]:
model.add(LSTM(32, return_sequences=True,input_shape=(X.shape[1],X.shape[2])))  # returns a sequence of vectors of dimension 32
model.add(LSTM(32, return_sequences=True))  # returns a sequence of vectors of dimension 32
model.add(LSTM(32))  # return a single vector of dimension 32
model.add(Dense(1, activation='softmax'))

In [103]:
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X,Y, epochs,1, verbose=2)

Epoch 1/1
 - 59s - loss: nan


In [105]:
print(model.summary(90))

__________________________________________________________________________________________
Layer (type)                            Output Shape                        Param #       
lstm_15 (LSTM)                          (None, 4, 32)                       5248          
__________________________________________________________________________________________
lstm_16 (LSTM)                          (None, 4, 32)                       8320          
__________________________________________________________________________________________
lstm_17 (LSTM)                          (None, 32)                          8320          
__________________________________________________________________________________________
dense_4 (Dense)                         (None, 1)                           33            
Total params: 21,921
Trainable params: 21,921
Non-trainable params: 0
__________________________________________________________________________________________
None


https://datascience.stackexchange.com/questions/26366/training-an-rnn-with-examples-of-different-lengths-in-keras

In [104]:
## Model 3

In [ ]:
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))